# Importing libraries

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import requests
import io
import re
import warnings
warnings.filterwarnings("ignore", category=UserWarning) # only to hide deprecation warning about axes.hold

The libraries being dealt with primarily are pandas, networkx, matplotlib, and requests. We are using pandas for the purpose of data analysis and bringing the data we'll be using into Python.

# Remove me possibly... globe!

plt.figure(figsize=(20,10))
world_map = Basemap(projection='robin',lon_0=0,resolution='l')
world_map.drawcountries(linewidth = 0.5)
world_map.fillcontinents(color='white',lake_color='white')
world_map.drawcoastlines(linewidth=0.5)

In the above code, we have a globe that we are using for mapping the languages onto. This code actually might be removed at the suggestion of the group as it might be too far reaching. The globe's size was initially small, so it was increased using the `plt.figure()` function. From there, Basemap was used to create a Robinson projection of the globe, and the countries and coasts were all drawn with 0.5 pixel line widths; the lakes were drawn in white to essentially make them invisible.

# Setting up the data

In [2]:
data_folder = "https://github.com/gabartomeo/data620-cunysps/raw/master/Assignment 02/Data/"
languoid_url = data_folder + "languoid.csv?raw=true"
languoid_content = requests.get(languoid_url).content
languoid_csv = pd.read_csv(io.StringIO(languoid_content.decode('utf-8')))
languoid = languoid_csv[["name", "id", "newick", "level"]].copy()
# print(languoid.head())
# print(languoid.describe())

The data we used came from [Kaggle](https://www.kaggle.com/rtatman/world-language-family-map). Using requests we gathered the content of the data stored on Github, and then read the entire file as a \*.CSV using the pandas library's `read_csv()` function. We sectioned out the columns we wanted - name, latitude, longitude, newick, and level - and copied it into a new variable. This list might need to be shortened depending on whether the group thinks we should try mapping it on the globe or not.

In [3]:
def strip_parentheses(newick_list):
    list_new = re.sub("\([^()\"]*(?:\"[^\"]*\"[^()\"]*)*\)", "", newick_list)
    return list_new

def remove_all_parentheses(newick_list_new):
    while True:
        try:
            list_newest = strip_parentheses(list_newest)
        except NameError:
            list_newest = strip_parentheses(newick_list_new)
        if "(" not in list_newest:
            break
    list_newest = re.findall("[a-z]{4}[0-9]{4}", list_newest)
    return list_newest

languoid["newick"] = languoid["newick"].map(lambda x: x[x.index("(")+1:x.rindex(")")] if x[0] == "(" else x)
languoid["newick"] = languoid["newick"].map(remove_all_parentheses)
languoid["newick"]

0                            [aala1237]
1                            [aant1238]
2                  [aari1239, gayi1237]
3                            [aari1239]
4                            [aari1240]
5                            [aasa1238]
6                            [aata1238]
7                            [abaa1238]
8                            [abab1239]
9                  [bali1274, oldb1234]
10                           [abad1240]
11                           [abad1241]
12                           [abag1245]
13                           [abai1239]
14                           [abai1240]
15                           [abai1241]
16                           [abak1242]
17                           [abak1243]
18                           [abal1238]
19                           [abam1239]
20                           [aban1242]
21                           [aban1243]
22       [aban1242, ekaj1238, nnam1238]
23                           [abar1238]
24                           [abau1245]


Two functions were created for the purpose of getting all the child languages, dialects, and families of a given language into a list format.

In [4]:
G = nx.Graph()
G.add_nodes_from(languoid["id"].tolist())

In [5]:
languoid["length_newick"] = languoid["newick"].apply(len)

In [6]:
languoid_pairs = languoid[languoid["length_newick"]>1].copy()
languoid_pairs = languoid_pairs[["id", "newick"]]
languoid_pairs["edges"] = languoid_pairs.apply(lambda x: [(x["id"], y) for y in x["newick"]], axis=1)
languoid_edges = []
for pairs in languoid_pairs["edges"].tolist():
    if type(pairs) == type([]):
        for each_edge in pairs:
            languoid_edges.append(each_edge)
    else:
        languoid_edges.append(pairs)
G.add_edges_from(languoid_edges)

lingo_unique = list(set(list(sum(languoid_edges, ()))))
lingo_edges_decon = list(sum(languoid_edges, ()))
temp = [lingo_edges_decon.count(x) for x in lingo_unique]
df = {"lingo": lingo_unique, "frequency": temp}
df = pd.DataFrame(data=df)
df.sort_values("frequency", ascending=False)

In [7]:
#pos=nx.spring_layout(G) #BIGTIME ERROR!

C:\Users\Lil Star\Anaconda3\lib\site-packages\networkx\drawing\layout.py:499: RuntimeWarning: invalid value encountered in sqrt
  distance = np.sqrt((delta**2).sum(axis=0))
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Lil Star\Anaconda3\lib\site-packages\networkx\drawing\layout.py", line 353, in fruchterman_reingold_layout
    dim, random_state)
  File "<decorator-gen-601>", line 2, in _sparse_fruchterman_reingold
  File "C:\Users\Lil Star\Anaconda3\lib\site-packages\networkx\utils\decorators.py", line 387, in _random_state
    return func(*new_args, **kwargs)
  File "C:\Users\Lil Star\Anaconda3\lib\site-packages\networkx\drawing\layout.py", line 506, in _sparse_fruchterman_reingold
    (delta * (k * k / distance**2 - Ai * distance / k)).sum(axis=1)
  File "C:\Users\Lil Star\Anaconda3\lib\site-packages\numpy\core\_methods.py", line 32, in _sum
    return umr_sum(a, axis, dtype, out, keepdims)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Lil Star\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.u

KeyboardInterrupt: 